In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from collections import defaultdict
from datetime import datetime
import h5py
import matplotlib.pyplot as plt
from neural_analysis.matIO import loadmat
import numpy as np
import os
import pandas as pd
import re
from scipy.signal import butter, filtfilt, freqz
import shutil
from sklearn.metrics import r2_score
import sys
import time
from tqdm.auto import tqdm

sys.path.append('..')
from utils import compile_folder, get_data_class, get_result_path, load_window_from_chunks, load, save, save_lfp_chunks

## Multiprocessing

In [3]:
all_data_dir = f"/om/user/eisenaj/datasets/anesthesia/mat"
task_type = 'VAR'
data_class = 'propofolPuffTone'
windows = [int(w) if w % 1 == 0 else w for w in np.arange(0.5, 10.5, 0.5)]
# bandpass_info = dict(
#     flag=False,
#     low=None,
#     high=None
# )
bandpass_info = dict(
    flag=True,
    low=0.5,
    high=100
)

In [4]:
print(f"BANDPASS = {bandpass_info['flag']}")
for session in [f[:-4] for f in os.listdir(os.path.join(all_data_dir, data_class)) if f.endswith('.mat')][:1]:
# for session in ['MrJones-Anesthesia-20160107-01']:
# for session in ['SPOCK_Ketamine_20210909']:
    data_class = get_data_class(session, all_data_dir)
    print("="*25)
    print(f"session = {session}")
    print("-"*25)
#     

    regex = re.compile(f"{session}_lfp_chunked_.*")
    data_dir = None
    chunk_time = np.Inf
    for f in os.listdir(os.path.join(all_data_dir, data_class)):
        if regex.match(f):
            if int(f.split('_')[-1][:-1]) < chunk_time:
                data_dir = os.path.join(all_data_dir, data_class, f)
                chunk_time = int(f.split('_')[-1][:-1])
    if data_dir is None:
        print("session has not been chunked, chunking now")
        save_lfp_chunks(session, all_data_dir)
    else:
        print("session has been chunked")
    
    
    results_dir = f"/om/user/eisenaj/ChaoticConsciousness/results/{data_class}/{task_type}"
    windows_to_run = []
    for window in windows:
        try:
            get_result_path(results_dir, session, window, bandpass_info=bandpass_info)
        except:
            windows_to_run.append(window)
            
    if len(windows_to_run) == 0:
        print(f"All windows have been run: {windows}")
    else:
        print(f"Now running: {windows_to_run}")
    
        for window in windows_to_run:
            stride = window
            os.system(f"sbatch /om2/user/eisenaj/code/ChaoticConsciousness/shell_scripts/multiproc.sh {session} {window} {stride} {task_type} {1 if bandpass_info['flag'] else 0} {bandpass_info['low']} {bandpass_info['high']}")

BANDPASS = True
session = MrJones-Anesthesia-20160105-01
-------------------------
session has been chunked
Now running: [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10]
Submitted batch job 23782808
Submitted batch job 23782809
Submitted batch job 23782810
Submitted batch job 23782811
Submitted batch job 23782812
Submitted batch job 23782813
Submitted batch job 23782814
Submitted batch job 23782815
Submitted batch job 23782816
Submitted batch job 23782817
Submitted batch job 23782835
Submitted batch job 23782836
Submitted batch job 23782837
Submitted batch job 23782838
Submitted batch job 23782839
Submitted batch job 23782840
Submitted batch job 23782841
Submitted batch job 23782842
Submitted batch job 23782843
Submitted batch job 23782844


## Fix Broken Compile

In [ ]:
dt = 0.001

In [35]:
session = 'Mary-Anesthesia-20160912-02'
data_class = get_data_class(session, all_data_dir)
results_dir = f"/om/user/eisenaj/ChaoticConsciousness/results/{data_class}/{task_type}"
for window in windows:
    result_path = get_result_path(results_dir, session, window, window)
    for folder_name in os.listdir(result_path):
        folder_to_compile = os.path.join(result_path, folder_name)
        if os.path.isdir(os.path.join(result_path, folder_to_compile)):
            files = os.listdir(folder_to_compile)
            print(f"Now compiling {folder_to_compile}")
            if len(files[0].split('_')) == 5:
                temp_data = load(os.path.join(folder_to_compile, files[0]))
                if 'start_ind' in temp_data.keys():
                    compile_folder(folder_to_compile)
                else:
                    all_window_data = []
                    for file in tqdm(files):
                        window_start, window_end = float(file.split('_')[2]), float(file.split('_')[4])
                        window_data = load(os.path.join(folder_to_compile, file))
                        window_data['start_time'] = window_start
                        window_data['start_ind'] = int(window_start/dt)
                        window_data['end_time'] = window_end
                        window_data['end_ind'] = int(window_end/dt)
                        all_window_data.append(window_data) 

                    results_df = pd.DataFrame(all_window_data)

                    results_df = results_df.sort_values('start_ind')
                    results_df = results_df.reset_index(drop=True)

                    shutil.rmtree(folder_to_compile)

                    save(results_df, folder_to_compile)
            else:
                compile_folder(folder_to_compile)

In [19]:
session = 'MrJones-Anesthesia-20160301-01'
window = 9.5
stride = 9.5
bandpass_info = {'flag': True, 'low': 0.5, 'high': 100}

In [7]:
results_dir = "/om/user/eisenaj/ChaoticConsciousness/results/propofolPuffTone/VAR"

In [15]:
file_template = f"{os.path.basename(results_dir)}_{session}_window_{window}_stride_{stride}_" + "[a-zA-Z]{3}-"
regex = re.compile(file_template)

In [16]:
matching_files = []
for file in os.listdir(results_dir):
    if regex.match(file):
        matching_files.append(file)

In [21]:
get_result_path(results_dir, session, window, bandpass_info=None)

'/om/user/eisenaj/ChaoticConsciousness/results/propofolPuffTone/VAR/VAR_MrJones-Anesthesia-20160301-01_window_9.5_stride_9.5_Feb-23-2022_2121'